Importar Librerías necesarias

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
import gc

Funciones creadas por los autores originales

In [ ]:
tf.__version__

In [ ]:
from utils import *
from GCIT import *
from GCIT_1 import *
import time
from csv import writer

In [ ]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [ ]:
%%time
x, y, z = generate_samples_random(size=500, sType='CI',dx=1, dy=1, dz=500,
                                  fixed_function='nonlinear', dist_z='gaussian')
p_vals = GCIT_1(x, y, z, statistic='all')
#print("Tiempo de ejecución: " + str(time.time()-ti))
p_vals

### Función No Lineal Con distribución Laplace

In [ ]:
Zdim = 500 # 25, 50, 100, 250, 500
dim_x = 1
dim_y = 1 
# fijar número de experimentos a realizar
n_experiments = 100 #faltan 900
# variable para guardar los pvalores de la función GCIT-1
p_vals= []
p_values = []
# determinar de manera aleatoria cual es el tipo de conjunto a evaluar
true = np.random.randint(0,2,n_experiments)
print('Probando test para '+str(Zdim)+' dimensiones de Z')
pbar = tqdm(total=n_experiments)
for n in range(n_experiments):
    ti = time.time()
    # CREAR DATASET
    if true[n]==0:
        data_type = 'CI' # conjunto condicionalmente independiente
    if true[n]==1:
        data_type = 'NI' # conjunto dependiente 
    x, y, z = generate_samples_random(size=500, sType=data_type,
                                      dx=dim_x, dy=dim_y, dz=Zdim,
                                      fixed_function='nonlinear', dist_z='laplace')
    # REALIZAR TEST
    p_vals = GCIT_1(x, y, z, statistic='all')
    del x, y, z
    pbar.update(1)
    # GUARDAR RESULTADOS
    append_list_as_row('dim%s_nonlinear.csv'%(Zdim,), [true[n].tolist()] + p_vals)
    gc.collect()
p_values = np.array(p_values)
pbar.close()

### Función Lineal Con distribución Gaussiana

In [ ]:
Zdim = 50 # 25, 50, 100, 250, 500
dim_x = 1
dim_y = 1 
# fijar número de experimentos a realizar
n_experiments = 3 #faltan 900
# variable para guardar los pvalores de la función GCIT-1
p_vals= []
p_values = []
# determinar de manera aleatoria cual es el tipo de conjunto a evaluar
true = np.random.randint(0,2,n_experiments)
print('Probando test para '+str(Zdim)+' dimensiones de Z')
pbar = tqdm(total=n_experiments)
for n in range(n_experiments):
    ti = time.time()
    # CREAR DATASET
    if true[n]==0:
        data_type = 'CI' # conjunto condicionalmente independiente
    if true[n]==1:
        data_type = 'NI' # conjunto dependiente 
    x, y, z = generate_samples_random(size=500, sType=data_type,
                                      dx=dim_x, dy=dim_y, dz=Zdim,
                                      fixed_function='linear', dist_z='gaussian')
    # REALIZAR TEST
    p_vals = GCIT_1(x, y, z, statistic='all')
    del x, y, z
    pbar.update(1)
    # GUARDAR RESULTADOS
    append_list_as_row('dim%s_linear_gaussian.csv'%(Zdim,), [true[n].tolist()] + p_vals)
    gc.collect()
p_values = np.array(p_values)
pbar.close()

### Función Lineal Con distribución Laplace

In [ ]:
Zdim = 25
dim_x = 1
dim_y = 1 
# fijar número de experimentos a realizar
n_experiments = 100
# variable para guardar los pvalores de la función GCIT-1
p_vals= []
p_values = []
# determinar de manera aleatoria cual es el tipo de conjunto a evaluar
true = np.random.randint(0,2,n_experiments)
print('Probando test para '+str(Zdim)+' dimensiones de Z')
pbar = tqdm(total=n_experiments)
for n in range(n_experiments):
    # CREAR DATASET
    if true[n]==0:
        data_type = 'CI' # conjunto condicionalmente independiente
    if true[n]==1:
        data_type = 'NI' # conjunto dependiente 
    x, y, z = generate_samples_random(size=500, sType=data_type,
                                      dx=dim_x, dy=dim_y, dz=Zdim,
                                      fixed_function='linear', dist_z='laplace')
    p_vals = GCIT_1(x, y, z, statistic='all')
    del x, y, z
    append_list_as_row('pvals_dim25_linear_laplace.csv', p_vals)
    append_list_as_row('trues_dim25_linear_laplace.csv', [true[n].tolist()])
    pbar.update(1)
    p_values.append(np.array(p_vals))
    gc.collect()
pbar.close()
p_values = np.array(p_values)
#np.savetxt('pvals_dim25_linear_laplace.csv', p_values, delimiter=';', fmt='10.5f')
#np.savetxt('trues_dim25_linear_laplace.csv', true, delimiter=',', fmt='%10.5f')

# FIN

In [ ]:
Zdims = [25,50,100,250,500]
dim_x = 1
dim_y = 1
alpha = 0.05
trues = []
predicts = []
p_values = []
n_experiments = 10
for dim_z in Zdims:
    times, p_vals = [], []
    true = np.random.randint(0,2,n_experiments)
    predict = []
    print('Probando test para '+str(dim_z)+' dimensiones de Z')
    pbar = tqdm(total=n_experiments)
    for n in range(n_experiments):
        # CREAR DATASET
        if true[n]==0:
            data_type = 'CI' # conjunto condicionalmente independiente
        if true[n]==1:
            data_type = 'NI' # conjunto dependiente 
        x, y, z = generate_samples_random(size=500, sType=data_type,
                                          dx=dim_x, dy=dim_y, dz=dim_z,
                                          fixed_function='nonlinear', dist_z='gaussian')
        p_val,_,_,_ = GCIT(x, y, z, statistic='corr')
        p_vals.append(p_val)
        if p_val < alpha/2.0:
            predict.append(1)
        else:
            predict.append(0)
        pbar.update(1)
    predicts.append(np.array(predict))
    trues.append(np.array(true))
    p_values.append(np.array(p_vals))


In [ ]:
p_vals